In [1]:
from liberate import fhe
from liberate.fhe import presets
import numpy as np

# Generate Engine with paramerers

In [2]:
params = presets.params["silver"]
print(params)
engine = fhe.ckks_engine(**params, verbose=True)

{'logN': 15, 'num_special_primes': 2, 'devices': [0], 'scale_bits': 40, 'num_scales': None}
[2023-11-20 02:06:51.575845] I have received the context:


I have received inputs:
        buffer_bit_length		= 62
        scale_bits			= 40
        logN				= 15
        N				= 32,768
        Number of special primes	= 2
        Number of scales		= 16
        Cache folder			= '/home/hanyul/.pyenv/versions/liberate/lib/python3.11/site-packages/liberate/fhe/cache/resources'
        Security bits			= 128
        Quantum security model		= post_quantum
        Security sampling distribution	= uniform
        Number of message bits		= 60
        In total I will be using '821' bits out of available maximum '829' bits.
        And is it secured?		= True
My RNS primes are [1099510054913, 1099515691009, 1099507695617, 1099516280833, 1099506515969, 1099520606209, 1099504549889, 1099523555329, 1099503894529, 1099527946241, 1099503370241, 1099529060353, 1099498258433, 1099531223041, 1099469684737, 109953200

# Set number of parties

In [3]:
num_of_parties = 5

# Generate test data

In [4]:
amin, amax = -256., 255.
m = engine.example(amin=amin, amax=amax)

# Create each party's Secret key

In [5]:
############################
####    generate sks    ####
############################
sks = [engine.create_secret_key() for _ in range(num_of_parties)]

# Create each party's Public key with CRS `a`

In [6]:
############################
####    generate pks    ####
############################
pks = [engine.create_public_key(sk=sks[0])]
crs = engine.clone(pks[0]).data[1]
for sk in sks[1:]:
    pks.append(engine.multiparty_create_public_key(sk, a=crs))

# Generate CPK(Collective Public Key)

In [7]:
cpk = engine.multiparty_create_collective_public_key(pks=pks)

# Create each party's galois key with CRS `a`

In [8]:
galks = [engine.create_galois_key(sks[0])]
crss = engine.generate_galois_crs(galks[0])
for sk in sks[1:]:
    galks.append(engine.multiparty_create_galois_key(sk=sk, a=crss))

# Genrate CGALK(Collective Galois key).

In [9]:
cgalk = engine.multiparty_generate_galois_key(galks)

# Encrypt data with CPK

In [10]:
ct = engine.encorypt(m, cpk)

# Rotation with galk & delta

In [11]:
# delta = np.random.randint(-engine.num_slots*2, engine.num_slots*2)
# delta = np.random.randint(-engine.num_slots, engine.num_slots)
delta = np.random.randint(-10, 10)
ct_rotated = engine.rotate_galois(ct, gk=cgalk, delta=delta)

# Decrypt using each party's secret key and Decode

In [12]:
pcts = [engine.multiparty_decrypt_head(ct_rotated, sks[0])]
for sk in sks[1:]:
    pcts.append(engine.multiparty_decrypt_partial(ct_rotated, sk))
m_ = engine.multiparty_decrypt_fusion(pcts, level=ct_rotated.level)

# Check the results and errors

In [13]:
idx = 10
print(f"delta : {delta}", "====="*17)
print(f"| idx |\t    original\t|\trolled\t  |\tencrypted   |\t  error\t    | idx |")
print("====="*17)
for idx, (x, y, z) in enumerate(zip(m[:idx], np.roll(m, delta)[:idx], m_[:idx])):
    print(f"| {idx:3d} |\t{x.real:15.9f} | {y.real:15.9f} | {z.real:15.9} | {(z-y).real:13e} | {idx-delta:>3d} |")
print("====="*17)
print(f">>\tABS max error {np.abs(np.roll(m, delta) - m_).max()}")
print("====="*17)

delta : 9 =====================================================================================
| idx |	    original	|	rolled	  |	encrypted   |	  error	    | idx |
|   0 |	 -193.318424185 |    34.818119550 |      34.8181195 |  4.840217e-11 |  -9 |
|   1 |	  129.081521490 |   102.798034971 |      102.798035 | -1.254818e-11 |  -8 |
|   2 |	   60.165638591 |  -219.147357810 |     -219.147358 | -5.400125e-12 |  -7 |
|   3 |	  -62.414612296 |   225.176963448 |      225.176963 |  5.110223e-11 |  -6 |
|   4 |	  175.941739747 |   195.368883493 |      195.368883 | -5.178435e-11 |  -5 |
|   5 |	 -106.448713128 |  -183.902003065 |     -183.902003 |  7.813128e-11 |  -4 |
|   6 |	  159.900464849 |  -144.282055301 |     -144.282055 | -7.574386e-11 |  -3 |
|   7 |	  201.451774585 |    31.183065478 |      31.1830655 | -3.209877e-11 |  -2 |
|   8 |	  247.623069234 |   135.672543618 |      135.672544 |  5.252332e-11 |  -1 |
|   9 |	  132.413022439 |  -193.318424185 |     -193.318424 |  7.460699e-11 |   

# Test Each each level's error

In [14]:
for level in range(engine.num_levels):

    ###################
    ####    enc    ####
    ###################
    ct = engine.encorypt(m, cpk, level=level)

    ######################
    ####    galois    ####
    ######################
    delta = np.random.randint(-10, 10)
    ct_rotated = engine.rotate_galois(ct, gk=cgalk, delta=delta)
    
    ###################
    ####    dec    ####
    ###################
    pcts = [engine.multiparty_decrypt_head(ct_rotated, sks[0])]
    for sk in sks[1:]:
        pcts.append(engine.multiparty_decrypt_partial(ct_rotated, sk))
    m_ = engine.multiparty_decrypt_fusion(pcts, level=ct_rotated.level)

    idx = 10
    print("====="*17)
    print(f"| level : {level:2d} ",  "====="*11, f"delta : {delta:2d}  |")
    print(f"| idx |\t    original\t|\trolled\t  |\tencrypted   |\t  error\t    | idx |")
    print("====="*17)
    for idx, (x, y, z) in enumerate(zip(m[:idx], np.roll(m, delta)[:idx], m_[:idx])):
        print(f"| {idx:3d} |\t{x.real:15.9f} | {y.real:15.9f} | {z.real:15.9} | {(z-y).real:13e} | {idx-delta:>3d} |")
    print("====="*17)
    print(f">>\tABS max error {np.abs(np.roll(m, delta) - m_).max()}")
    print("====="*17, end="\n\n")

| level :  0  ======================================================= delta :  0  |
| idx |	    original	|	rolled	  |	encrypted   |	  error	    | idx |
|   0 |	 -193.318424185 |  -193.318424185 |     -193.318424 |  5.755396e-11 |   0 |
|   1 |	  129.081521490 |   129.081521490 |      129.081521 | -1.112710e-10 |   1 |
|   2 |	   60.165638591 |    60.165638591 |      60.1656386 | -6.042455e-11 |   2 |
|   3 |	  -62.414612296 |   -62.414612296 |     -62.4146123 | -1.163869e-11 |   3 |
|   4 |	  175.941739747 |   175.941739747 |       175.94174 |  7.185008e-11 |   4 |
|   5 |	 -106.448713128 |  -106.448713128 |     -106.448713 |  4.736478e-11 |   5 |
|   6 |	  159.900464849 |   159.900464849 |      159.900465 |  6.389200e-11 |   6 |
|   7 |	  201.451774585 |   201.451774585 |      201.451775 |  4.544631e-11 |   7 |
|   8 |	  247.623069234 |   247.623069234 |      247.623069 | -3.959144e-11 |   8 |
|   9 |	  132.413022439 |   132.413022439 |      132.413022 | -1.194849e-10 |   9 |
>>	ABS m

# Test rotate sum

In [15]:
ct = engine.encorypt(m, cpk)
ct_sum = engine.sum(ct, gk=cgalk, rescale_every=5)

pcts = [engine.multiparty_decrypt_head(ct_sum, sks[0])]
for sk in sks[1:]:
    pcts.append(engine.multiparty_decrypt_partial(ct_sum, sk))
m_ = engine.multiparty_decrypt_fusion(pcts, level=ct_sum.level)

idx = 10
print(f"\torigin\t|\t   encrypted\t|\t\terror\t|")
print(f"{sum(m).real:15.8f}\t|\t{m_[0].real:15.8f}\t|\t{(m_[0] - sum(m)).real:13.4e}\t|")

	origin	|	   encrypted	|		error	|
 -3637.63057476	|	 -3637.63057477	|	  -7.2801e-09	|
